In [1]:
from pathlib import Path
import glob
import np as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from biopsykit.utils.dataframe_handling import multi_xs

/Users/Philipp/git/ba-philipp-doerfler/Code/respiration_extraction/.venv/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#Function for loading all data
PATH = Path("/Users/Philipp/git/ba_resp_dataset/subjects/philipp_dataset")
def load_csvs():
    all_csvs = glob.glob( (PATH / "**/wide/*.csv").__str__() , recursive=True)
    dfs = []
    for csv in all_csvs:
        df = pd.read_csv(csv)
        dfs.append(df)

    final_df = pd.concat(dfs, axis=0)
    return final_df

In [3]:
#Load all data into final dataframe and bring it to long format
#How to access values:
#final.loc[(study,participant,phases,extraction, estimation, sensor, measurement),:]
#Multiindex parameter can be given as:
#   single value: if for example only one participant is wanted
#   list of values: if for example multiple participants are wanted
#   slice(None): if all participants are wanted
final = load_csvs()
del final["Unnamed: 0"]

index_cols = [
        "Study",
        "Participant",
        "Phase",
        "Extraction",
        "Estimation",
        "Sensor",
    ]

final.reset_index(drop=True)
final = final.set_index(index_cols)
final.columns = pd.MultiIndex.from_product([["Values"],final.columns])
final = final.stack()
#Rename column containing the values for evaluation
final.index.rename({None:"Measurement"},inplace=True)
final

Values
Study      Participant Phase             Extraction          Estimation           Sensor      Measurement           
BA_Philipp 12          Baseline 3        ExtractionKarlen    PeakDetection        Biopac ECG  CC           -0.176658
                                                                                              Difference   -6.569521
                                                                                              GT           17.240002
                                                                                              RR           23.809524
                                                             PeakThroughDetection Biopac ECG  CC           -0.176658
...                                                                                                              ...
           5           Standing and rest SavGolExtractionAcc CountAdvDetection    NilsPod IMU RR            6.831571
                                                             GradiantDetection    NilsPod IMU CC           -0.505340
                                                                                              Difference    0.149608
                                                                                              GT            8.155863
                                                                                              RR            8.006255

[63720 rows x 1 columns]

In [4]:
#Testing Final Data
print(type(final))
print(final)
print(final.index)



<class 'pandas.core.frame.DataFrame'>
                                                                                                              Values
Study      Participant Phase             Extraction          Estimation           Sensor      Measurement           
BA_Philipp 12          Baseline 3        ExtractionKarlen    PeakDetection        Biopac ECG  CC           -0.176658
                                                                                              Difference   -6.569521
                                                                                              GT           17.240002
                                                                                              RR           23.809524
                                                             PeakThroughDetection Biopac ECG  CC           -0.176658
...                                                                                                              ...
           5           Sta

# Calculate Statistics


In [5]:
#Define Lists of Phases and Algorithms
relevant_phases= [
        #"Initialization",
        "Baseline 1",
        "Speaking",
        "Baseline 2",
        "Standing and rest",
        "Sitting and rest",
        "4-7-8 breathing",
        "Baseline 3",
        "Metronome Breathing",
        "Baseline 4",
        "Inhalation and hold",
        "Exhalation and hold",
        "Hyperventilation",
    ]
ecg_extractions = [
    "ExtractionKarlen",
    "ExtractionCharlton",
    'ExtractionVangent2019',
    "ExtractionLindeberg",
    "ExtractionSoni2019",
    "ExtractionAddisonAM",
    "ExtractionAddisonFM",
    "ExtractionOrphandiou",
    "ExtractionSarkar2015",
]
imu_extractions = ["PositionalVectorExtraction", "SavGolExtractionGyr", "SavGolExtractionAcc"]
estimations = [
    "PeakDetection",
    "PeakThroughDetection",
    "CountOrig",
    "CountAdvDetection",
    "GradiantDetection",
]
fusion = ["SmartFusion"]


In [6]:
#Calculate Mean Value of Ground Truth and Estimated Respiration Rate
# Mean ground truth respiration rate over all phases
gt_rr_mean = final.loc["BA_Philipp",:,relevant_phases,:,: ,:, "GT"]["Values"].mean()
# Mean estimated respiratory rate
estimated_rr_mean =  final.loc["BA_Philipp",:,relevant_phases,:,: ,:, "RR"]["Values"].mean()
print(gt_rr_mean,estimated_rr_mean)

17.45229337567751 284.7516838799964


In [7]:
#Select results from the relevant phases
rrs = final.loc["BA_Philipp",:,relevant_phases,:,:,:,"RR"]
gts = final.loc["BA_Philipp",:,relevant_phases,:,:,:,"GT"]
differences = final.loc["BA_Philipp",:,relevant_phases,:,:,:,"Difference"]
ccs =final.loc["BA_Philipp",:,relevant_phases,:,:,:,"CC"]
rrs

Values
Study      Participant Phase            Extraction          Estimation           Sensor      Measurement            
BA_Philipp 12          Baseline 1       ExtractionKarlen    PeakDetection        Biopac ECG  RR            20.113179
                                                            PeakThroughDetection Biopac ECG  RR             3.409013
                                                            CountOrig            Biopac ECG  RR            16.400081
                                                            CountAdvDetection    Biopac ECG  RR            10.705001
                                                            GradiantDetection    Biopac ECG  RR            39.473684
...                                                                                                              ...
           5           Hyperventilation SavGolExtractionAcc PeakDetection        NilsPod IMU RR           117.223725
                                                            PeakThroughDetection NilsPod IMU RR             5.538129
                                                            CountOrig            NilsPod IMU RR            21.037494
                                                            CountAdvDetection    NilsPod IMU RR             9.983390
                                                            GradiantDetection    NilsPod IMU RR            16.599424

[15060 rows x 1 columns]

In [8]:
# BioPac Results
result_dataframe_wide = pd.DataFrame(
    columns=[
        "Extraction",
        "Estimation",
        "Correlation Mean",
        "Correlation STD",
        "RR Mean Absolute Error",
    ]
)
for extraction in ecg_extractions:
    for estimation in estimations:
        biopac_differences = differences.loc[:,:,:,extraction,estimation,"Biopac ECG",:]["Values"]
        biopac_ccs = ccs.loc[:,:,:,extraction,estimation,"Biopac ECG",:]["Values"]
        # Mean absolute Error
        algorithm_mean_error = abs(biopac_differences).mean()
        # Mean Coefficients with STD
        algorithm_cc_mean= biopac_ccs.mean()
        algorithm_cc_std=biopac_ccs.std()

        row_dict = {
            "Extraction": extraction,
            "Estimation": estimation,
            "Correlation Mean": algorithm_cc_mean,
            "Correlation STD":algorithm_cc_std,
            "RR Mean Absolute Error": algorithm_mean_error
        }
        append = pd.DataFrame(row_dict,index=[0])
        result_dataframe_wide = pd.concat(
                [result_dataframe_wide, append], ignore_index=True
        )
biopac_results = result_dataframe_wide
biopac_results

,Extraction,Estimation,Correlation Mean,Correlation STD,RR Mean Absolute Error
0,ExtractionKarlen,PeakDetection,-0.314353,0.249254,9.84998
1,ExtractionKarlen,PeakThroughDetection,-0.314353,0.249254,8.37099
2,ExtractionKarlen,CountOrig,-0.314353,0.249254,7.869668
3,ExtractionKarlen,CountAdvDetection,-0.314353,0.249254,8.04716
4,ExtractionKarlen,GradiantDetection,-0.314353,0.249254,8.049854
5,ExtractionCharlton,PeakDetection,-0.255496,0.225916,11.385997
6,ExtractionCharlton,PeakThroughDetection,-0.255496,0.225916,8.557275
7,ExtractionCharlton,CountOrig,-0.255496,0.225916,9.003326
8,ExtractionCharlton,CountAdvDetection,-0.255496,0.225916,7.903433
9,ExtractionCharlton,GradiantDetection,-0.255496,0.225916,9.127045


In [9]:
#Create NilsPod ECG Results
result_dataframe_wide = pd.DataFrame(
    columns=[
        "Extraction",
        "Estimation",
        "Correlation Mean",
        "Correlation STD",
        "RR Mean Absolute Error",
    ]
)

for extraction in ecg_extractions:
    for estimation in estimations:
        nils_differences = differences.loc[:,:,:,extraction,estimation,"NilsPod ECG",:]["Values"]
        nils_ccs = ccs.loc[:,:,:,extraction,estimation,"NilsPod ECG",:]["Values"]
        # Mean absolute Error
        algorithm_mean_error = abs(nils_differences).mean()
        # Mean CC
        algorithm_cc_mean= nils_ccs.mean()
        algorithm_cc_std = nils_ccs.std()

        row_dict = {
            "Extraction": extraction,
            "Estimation": estimation,
            "Correlation Mean": algorithm_cc_mean,
            "Correlation STD":algorithm_cc_std,
            "RR Mean Absolute Error": algorithm_mean_error
        }
        append = pd.DataFrame(row_dict,index=[0])
        result_dataframe_wide = pd.concat(
                [result_dataframe_wide, append], ignore_index=True
        )

nilspod_ecg_results = result_dataframe_wide
nilspod_ecg_results

,Extraction,Estimation,Correlation Mean,Correlation STD,RR Mean Absolute Error
0,ExtractionKarlen,PeakDetection,0.038645,0.33661,9.17467
1,ExtractionKarlen,PeakThroughDetection,0.038645,0.33661,8.456003
2,ExtractionKarlen,CountOrig,0.038645,0.33661,7.777508
3,ExtractionKarlen,CountAdvDetection,0.038645,0.33661,8.265588
4,ExtractionKarlen,GradiantDetection,0.038645,0.33661,9.164712
5,ExtractionCharlton,PeakDetection,0.077111,0.304684,9.057889
6,ExtractionCharlton,PeakThroughDetection,0.077111,0.304684,8.463781
7,ExtractionCharlton,CountOrig,0.077111,0.304684,7.57107
8,ExtractionCharlton,CountAdvDetection,0.077111,0.304684,8.28529
9,ExtractionCharlton,GradiantDetection,0.077111,0.304684,8.652321


In [10]:
# Create IMU Results
result_dataframe_wide = pd.DataFrame(
    columns=[
        "Extraction",
        "Estimation",
        "Correlation Mean",
        "Correlation STD",
        "RR Mean Absolute Error",
    ]
)
for extraction in imu_extractions:
    for estimation in estimations:
        nils_differences = differences.loc[:,:,:,extraction,estimation,"NilsPod IMU",:]["Values"]
        nils_ccs = ccs.loc[:,:,:,extraction,estimation,"NilsPod IMU",:]["Values"]
        # Mean absolute Error
        algorithm_mean_error = abs(nils_differences).mean()
        # Mean CC
        algorithm_cc_mean= nils_ccs.mean()
        algorithm_cc_std = nils_ccs.std()

        row_dict = {
            "Extraction": extraction,
            "Estimation": estimation,
            "Correlation Mean": algorithm_cc_mean,
            "Correlation STD":algorithm_cc_std,
            "RR Mean Absolute Error": algorithm_mean_error
        }
        append = pd.DataFrame(row_dict,index=[0])
        result_dataframe_wide = pd.concat(
                [result_dataframe_wide, append], ignore_index=True
        )

nilspod_imu_results = result_dataframe_wide
nilspod_imu_results

,Extraction,Estimation,Correlation Mean,Correlation STD,RR Mean Absolute Error
0,PositionalVectorExtraction,PeakDetection,0.024278,0.304966,52.313423
1,PositionalVectorExtraction,PeakThroughDetection,0.024278,0.304966,11.643594
2,PositionalVectorExtraction,CountOrig,0.024278,0.304966,8.357204
3,PositionalVectorExtraction,CountAdvDetection,0.024278,0.304966,9.057994
4,PositionalVectorExtraction,GradiantDetection,0.024278,0.304966,11.808556
5,SavGolExtractionGyr,PeakDetection,-0.08638,0.186025,47.143896
6,SavGolExtractionGyr,PeakThroughDetection,-0.08638,0.186025,11.176994
7,SavGolExtractionGyr,CountOrig,-0.08638,0.186025,11.055842
8,SavGolExtractionGyr,CountAdvDetection,-0.08638,0.186025,8.748673
9,SavGolExtractionGyr,GradiantDetection,-0.08638,0.186025,14.851874


In [11]:
#Create Overall Statistics for correlation but delete correlation for latex table
results_overall = pd.concat([biopac_results,nilspod_ecg_results,nilspod_imu_results])

results_overall = results_overall.drop_duplicates(["Sensor","Extraction"])
results_overall.reset_index(drop=False)
results_overall.sort_values(by=["Correlation Mean"], key=abs)




KeyError: Index(['Sensor'], dtype='object')

In [ ]:

print(results_overall)
del results_overall["Correlation STD"]
del results_overall["Correlation Mean"]

results_overall = results_overall.reindex(columns=["Extraction", "Estimation", "Sensor", "RR Mean Absolute Error"])
results_overall = results_overall.sort_values(by="RR Mean Absolute Error")
print(results_overall)
latex = results_overall.to_latex()
filename="/Users/Philipp/git/ba-philipp-doerfler/Thesis/BA_Philipp_Doerfler/img/table.tex"
latex_file = open(filename , "w")
latex_file.write(latex)
latex_file.close()

In [ ]:
biopac_results["Sensor"] = "BioPac ECG"
biopac_results.set_index(["Sensor", "Extraction" , "Estimation"], append=True)

nilspod_ecg_results["Sensor"] = "NilsPod ECG"
nilspod_ecg_results.set_index(["Sensor", "Extraction" , "Estimation"], append=True)

nilspod_imu_results["Sensor"] = "NilsPod IMU"
nilspod_imu_results.set_index(["Sensor", "Extraction" , "Estimation"], append=True)

In [ ]:
#Boxplots of Extractions for sensor types
fig ,ax = plt.subplots()
data = final.loc[(slice(None),slice(None),["Metronome Breathing"],['ExtractionKarlen',"SavGolExtractionAcc"],slice(None),slice(None),"CC"),:].reset_index(level=["Sensor"])
print(data)
from fau_colors import cmaps
palette = cmaps.faculties_light
sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

# bio = final_wide.loc[:,:,:,:,:,"Biopac ECG"]["CC"]
# imu = final_wide.loc[:,:,:,:,:,"NilsPod IMU"]["CC"]
# nils = final_wide.loc[:,:,:,:,:,"NilsPod ECG"]["CC"]
#data.set_index(["Study" , "Participant", "Phase","Extraction","Estimation"])
# data = data.reset_index(drop=False)
sns.set_theme(context="notebook", style="ticks", palette=palette)
sns.violinplot(data=data, x="Sensor", y = "Values" , flierprops={"marker": "x"}, ax=ax)
#Speichern
file="/Users/Philipp/git/ba-philipp-doerfler/Thesis/BA_Philipp_Doerfler/img/SensorBoxPlots.pdf"
ax.set_ylabel("Pearson correlation coefficient")
ax.set_xlabel("Signal used for extraction")


fig.align_ylabels()
fig.align_xlabels()
fig.savefig(file)


In [ ]:
# CC of the best Algorithm (ExtractionKarlen) for all phases with violin Plots
fig,ax = plt.subplots( nrows=2,figsize=(10,10))
saturation = 1
# Filter df for best Combination and Phases
phases_plot1 =  [
        "Metronome Breathing",
        "Inhalation and hold",
        "Exhalation and hold",
        "Hyperventilation",]
phases_plot2 = [
        "Speaking",
        "Standing and rest",
        "Sitting and rest",
        "4-7-8 breathing",
        ]
data_phases1_4 = final.loc[(slice(None),slice(None),phases_plot1,"ExtractionKarlen","GradiantDetection","Biopac ECG","CC"),:].reset_index(level="Phase")
data_phases5_8 = final.loc[(slice(None),slice(None),phases_plot2,"ExtractionKarlen","GradiantDetection","Biopac ECG","CC"),:].reset_index(level="Phase")

sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)

sns.violinplot(data=data_phases1_4,cut=2,x ="Phase", y="Values",saturation=saturation, ax=ax[0])
sns.swarmplot(data=data_phases1_4, x="Phase", y="Values",ax=ax[0], palette=cmaps.faculties_dark)

sns.violinplot(data= data_phases5_8,cut=1, x ="Phase", y="Values",saturation=saturation, ax=ax[1])
sns.swarmplot(data= data_phases5_8, x="Phase", y="Values",ax=ax[1],palette=cmaps.faculties_dark)


file="/Users/Philipp/git/ba-philipp-doerfler/Presentations/final_talk/img/BestAlgoPhasesCC_ViolinPlots.pdf"
ax[0].set_ylabel("Pearson correlation coefficient")
ax[0].set_xlabel("Phases")
ax[0].xaxis.label.set_fontsize(15)
ax[0].yaxis.label.set_fontsize(15)
ax[0].set_ylim(-1,1)


ax[1].set_ylabel("Pearson correlation coefficient")
ax[1].set_xlabel("Phases")
ax[1].xaxis.label.set_fontsize(15)
ax[1].yaxis.label.set_fontsize(15)
ax[1].set_ylim(-1,1)
fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)

In [ ]:
# CC of the best Algorithm (ExtractionKarlen) for all phases with Box Plots
fig,ax = plt.subplots( nrows=2,figsize=(10,10))
saturation = 1
# Filter df for best Combination and Phases
phases_plot1 =  [
        "Metronome Breathing",
        "Inhalation and hold",
        "Exhalation and hold",
        "Hyperventilation",]
phases_plot2 = [
        "Speaking",
        "Standing and rest",
        "Sitting and rest",
        "4-7-8 breathing",
        ]
data_phases1_4 = final.loc[(slice(None),slice(None),phases_plot1,"ExtractionKarlen","GradiantDetection",slice(None),"CC"),:].reset_index(level="Phase")
data_phases5_8 = final.loc[(slice(None),slice(None),phases_plot2,"ExtractionKarlen","GradiantDetection",slice(None),"CC"),:].reset_index(level="Phase")

sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)

sns.boxplot(data=data_phases1_4,x ="Phase", y="Values",saturation=saturation, ax=ax[0])
sns.swarmplot(data=data_phases1_4, x="Phase", y="Values",ax=ax[0], palette=cmaps.faculties_dark)

sns.boxplot(data=data_phases5_8,x ="Phase", y="Values",saturation=saturation, ax=ax[1])
sns.swarmplot(data= data_phases5_8, x="Phase", y="Values",ax=ax[1],palette=cmaps.faculties_dark)


file="/Users/Philipp/git/ba-philipp-doerfler/Thesis/BA_Philipp_Doerfler/img/BestAlgoPhasesCC_Boxplots.pdf"
ax[0].set_ylabel("Pearson correlation coefficient")
ax[0].set_xlabel("Phases")
ax[0].xaxis.label.set_fontsize(15)
ax[0].yaxis.label.set_fontsize(15)
ax[0].set_ylim(-1,1)


ax[1].set_ylabel("Pearson correlation coefficient")
ax[1].set_xlabel("Phases")
ax[1].xaxis.label.set_fontsize(15)
ax[1].yaxis.label.set_fontsize(15)
ax[1].set_ylim(-1,1)
fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)

# Respiration Rate Estimation

In [ ]:
# Find the best Algorithm for RR Estimation
# Get 2 Lists for each algorithm with all predicted Respiration rates and their Ground truth
residiual_data = pd.DataFrame(columns=["Extraction", "Estimation" ,"Sensor", "RR" , "GT"])

for extraction in ecg_extractions:
    for estimation in estimations:
        #Get RRs and GTs for the algorithm combination
        rrs = list(final.loc[("BA_Philipp", slice(None) , relevant_phases , extraction, estimation, "Biopac ECG", "RR"),:])
        gts = list(final.loc[("BA_Philipp", slice(None) , relevant_phases , extraction, estimation, "Biopac ECG", "GT"),:])
        #create df row
        residiual_dic = {"Extraction": extraction , "Estimation": estimation , "Sensor": "Biopac ECG" , "RR": np.nan, "GT": np.nan}
        append = pd.DataFrame(residiual_dic, index=[0])
        append["RR"] = append["RR"].astype('object')
        append["GT"] = append["GT"].astype('object')
        #Set the whole list as value in one field
        append.at[0,"RR"] = rrs
        append.at[0,"GT"] = gts
        residiual_data = pd.concat([residiual_data,append] ,ignore_index=True)

residiual_data = residiual_data.set_index(["Extraction", "Estimation"])
residiual_data.index = pd.MultiIndex.from_frame(pd.DataFrame(index=residiual_data.index).reset_index().astype(str))
residiual_data

## Rate estimation

In [ ]:
#Create Residual Plot for Fused RR Estimation Algorithm
fig,ax = plt.subplots(figsize = (8,8))
sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)

def fuse(*resp_rates: float):
    # Calculate Standarddeviation of all RRs
    std = np.std(resp_rates)
    # If std <= 4 bpm then estimate RR as the mean else no Output
    if std <= 4.0:
        respiration_rate_fused = np.mean(resp_rates)
    else:
        respiration_rate_fused = np.nan
    return respiration_rate_fused


ecg_datapoints = final.loc[(slice(None), slice(None),slice(None),"ExtractionSoni2019","CountOrig", "Biopac ECG", slice(None)),:]

imu_datapoints = final.loc[(slice(None), slice(None),slice(None),"ExtractionSarkar2015","CountOrig", "Biopac ECG", slice(None)),:]

nils_datapoints = final.loc[(slice(None), slice(None),slice(None),"ExtractionCharlton","CountOrig", "NilsPod ECG", slice(None)),:]
ad1 = final.loc[(slice(None), slice(None),slice(None),"PositionalVectorExtraction","CountOrig", "NilsPod IMU", slice(None)),:]
ad2= final.loc[(slice(None), slice(None),slice(None),"SavGolExtractionAcc","CountOrig", "NilsPod IMU", slice(None)),:]


dataframe = pd.DataFrame(columns=["Subject" , "Phase","RR", "GT"])
for subject in final.index.levels[1]:
    for phase in relevant_phases:
        rr_1 = ecg_datapoints.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),'RR'),:].iloc[0,0]
        rr_2 = imu_datapoints.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),'RR'),:].iloc[0,0]
        rr_3 = nils_datapoints.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),'RR'),:].iloc[0,0]
        rr_4 = ad1.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),'RR'),:].iloc[0,0]
        rr_5 = ad2.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),'RR'),:].iloc[0,0]
        fused = fuse(rr_1,rr_2,rr_3,rr_4,rr_5)
        try:
            gt = ecg_datapoints.loc[(slice(None),subject,phase,slice(None),slice(None),slice(None),"GT"),:].iloc[0,0]
        except KeyError:
            gt = np.nan

        if (fused == np.nan or gt == np.nan):
            print(fused,gt)
            continue
        dict = {"Subject": subject, "Phase": phase , "RR": fused, "GT" :gt}
        append = pd.DataFrame(dict, index=["0"])
        dataframe = pd.concat([dataframe,append])
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.dropna()
dataframe[["RR","GT"]]= dataframe[["RR","GT"]].astype(float)
print(dataframe)
#Create Plot
dataframe = dataframe.dropna()
sns.scatterplot( x=dataframe["GT"], y = (dataframe["GT"]- dataframe["RR"]))
ax.set_xlabel("Ground Truth RR [BPM]")
ax.set_ylabel("Ground Truth RR - Estimated RR [BPM]")
ax.plot([0,100],[0,0],'--')
file="/Users/Philipp/git/ba-philipp-doerfler/Presentations/final_talk/img/EstimationFusedNew.pdf"
fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)


In [ ]:
best_algo = data[
    data["Extraction"].isin(["ExtractionSoni2019"]) & data["Estimation"].isin(["CountOrig"]) & data["Sensor"].isin(
        ["Biopac ECG"])]
best_algo_phases = best_algo[best_algo["Phase"].isin(["Metronome Breathing",
                                                      "Inhalation and hold",
                                                      "Exhalation and hold",
                                                      "Hyperventilation",
                                                      "Speaking",
                                                      "Standing and rest",
                                                      "Sitting and rest",
                                                      "4-7-8 breathing", ])]

def create_table(df):
    return df.to_latex()

In [ ]:
# residualplot for all phases
# fig,ax =plt.subplots()
# sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)
# final_long = final_wide.dropna().stack()
#
# df = multi_xs(final_long, ["CC" , "GT"] , level="Performance Indicator")
# rrs = final_long.loc[:,:,:,:,:,"Biopac ECG", "RR"]
# gts = final_long.loc[:,:,:,:,:,"Biopac ECG", "GT"]
#
# points = list(zip(list(gts.iloc[:,0]),list(rrs.iloc[:,0])))
# points_filtered = [point for point in points if not (-100 > point[0] or point[0] > 100 or -100 > point[1] or point[1] >100)]
# x_filtered, y_filtered = zip(*points_filtered)
#
# sns.residplot(x=list(x_filtered), y =list(y_filtered), ax=ax)


# One Residual Plot for best Algorithm
fig,ax =plt.subplots(1,figsize=(8,8))
sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)
# Get datapoints
relevant_phases= [
        #"Initialization",
        #"Baseline 1",
        "Speaking",
        #"Baseline 2",
        "Standing and rest",
        "Sitting and rest",
        "4-7-8 breathing",
        #"Baseline 3",
        "Metronome Breathing",
        #"Baseline 4",
        "Inhalation and hold",
        "Exhalation and hold",
        "Hyperventilation",
    ]
datapoints = final.loc[(slice(None), slice(None),relevant_phases,"ExtractionSoni2019","CountOrig", "Biopac ECG", ["RR","GT"]),:].reset_index(level="Measurement")
df = pd.DataFrame()
df["rr"]=datapoints.Values[datapoints["Measurement"]=="RR"]
df["gt"]=datapoints.Values[datapoints["Measurement"]=="GT"]
df["diff"]=abs(df["gt"] - df["rr"])

sns.scatterplot(x=df["gt"], y= df["gt"] - df["rr"], ax=ax)
ax.plot([0,100],[0,0],'--')
#sns.residplot( data=df, x="gt", y="rr",order=0, ax=ax,dropna=True)

file="/Users/Philipp/git/ba-philipp-doerfler/Presentations/final_talk/img/EstimationBestAlgoNew_real.pdf"
ax.set_ylabel("Ground Truth RR - Estimated RR [BPM]")
ax.set_xlabel("Ground Truth RR [BPM]")
ax.xaxis.label.set_fontsize(15)
ax.yaxis.label.set_fontsize(15)

fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)

In [ ]:
# Residual Plot for each phase
fig,ax = plt.subplots(nrows=4,ncols=2, figsize=(10,20))
sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)
data_1 =  final.loc[(slice(None), slice(None),relevant_phases[:len(relevant_phases)//2],"ExtractionSoni2019","CountOrig", "Biopac ECG", ["RR","GT"]),:].reset_index(level="Measurement")
df = pd.DataFrame()
df["rr"]=data_1.Values[data_1["Measurement"]=="RR"]
df["gt"]=data_1.Values[data_1["Measurement"]=="GT"]
df["diff"]=abs(df["gt"] - df["rr"])

data_2 = final.loc[(slice(None), slice(None),relevant_phases[len(relevant_phases)//2:],"ExtractionSoni2019","CountOrig", "Biopac ECG", ["RR","GT"]),:].reset_index(level="Measurement")
df2 = pd.DataFrame()
df2["rr"]=data_2.Values[data_2["Measurement"]=="RR"]
df2["gt"]=data_2.Values[data_2["Measurement"]=="GT"]
df2["diff"]=abs(df2["gt"] - df2["rr"])

counter=0
for phase in relevant_phases[:len(relevant_phases)//2]:
    ax[counter,0].plot(df.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"gt"], df.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"gt"] - df.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"rr"] , 'o')
    #sns.residplot( data =df.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),:] , x="gt", y="rr", ax=ax[counter,0])
    ax[counter,0].set_title(phase)
    ax[counter,0].set_xlabel("Ground Truth RR [BPM]")
    ax[counter,0].set_ylabel("Ground Truth RR - Estimated RR [BPM]")
    ax[counter,0].set_ylim(-12.5, 12.5)
    print(phase, df.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),:].mean())
    counter +=1

counter=0
for phase in relevant_phases[len(relevant_phases)//2:]:
    ax[counter,1].plot(df2.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"gt"], df2.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"gt"] - df2.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),"rr"] , 'o')
    sns.residplot(data=df2.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),:],x="gt", y="rr", ax=ax[counter,1])

    ax[counter,1].set_title(phase)
    ax[counter,1].set_xlabel("Ground Truth RR [BPM]")
    ax[counter,1].set_ylabel("Ground Truth RR - Estimated RR [BPM]")
    ax[counter,1].set_ylim(-12.5, 12.5)
    print(phase, df2.loc[(slice(None),slice(None),phase,slice(None),slice(None),slice(None)),:].mean())
    counter +=1

file="/Users/Philipp/git/ba-philipp-doerfler/Presentations/final_talk/img/EstimationPhases_real.pdf"
fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)

In [ ]:
# Calculate the SmartFusion
fig,ax = plt.subplots(nrows=4,ncols=2, figsize=(10,20))
sns.set_theme(context="talk",  style="ticks",font = "sans-serif", palette=palette)
#Dataframe erstellen
dataframe["Diff"] = abs(dataframe["GT"] - dataframe["RR"])
print(dataframe)
counter=0
for phase in relevant_phases[:len(relevant_phases)//2]:
    sns.residplot(data=dataframe.loc[dataframe["Phase"] == phase], x ="GT", y = "RR", ax=ax[counter,0])
    print(phase, dataframe.loc[dataframe["Phase"] == phase]["Diff"].mean())
    ax[counter,0].set_title(phase)
    ax[counter,0].set_xlabel("Ground Truth RR [BPM]")
    ax[counter,0].set_ylabel("Ground Truth RR - Estimated RR [BPM]")
    ax[counter,0].set_ylim(-12.5, 12.5)
    counter+=1

counter=0
for phase in relevant_phases[len(relevant_phases)//2:]:
    sns.residplot(data=dataframe.loc[dataframe["Phase"] == phase], x ="GT", y = "RR", ax=ax[counter,1])
    print(phase, dataframe.loc[dataframe["Phase"] == phase]["Diff"].mean())
    ax[counter,1].set_title(phase)
    ax[counter,1].set_xlabel("Ground Truth RR [BPM]")
    ax[counter,1].set_ylabel("Ground Truth RR - Estimated RR [BPM]")
    ax[counter,1].set_ylim(-12.5, 12.5)
    counter +=1
file="/Users/Philipp/git/ba-philipp-doerfler/Presentations/final_talk/img/EstimationPhases_fused_new.pdf"
fig.align_ylabels()
fig.align_xlabels()
fig.tight_layout()
fig.savefig(file)

In [ ]:
#Create Big figure with residual plots for each algorithm
for extraction in ecg_extractions:
    for estimation in estimations:
        index1 = ecg_extractions.index(extraction)
        index2 = estimations.index(estimation)
        y= residiual_data.loc[extraction,estimation]["RR"]
        x = residiual_data.loc[extraction,estimation]["GT"]
        ax[index1,index2].set_title( extraction + " + " + estimation)
        ax[index1,index2].set_xlabel("Ground Truth Respiration Rate [BPM]")
        ax[index1,index2].set_ylabel("Estimated Respiration Rate - Ground Truth Respiration Rate [BPM]")
        sns.residplot(x=list(x),y=list(y),ax=ax[index1,index2])

fig.tight_layout()
file="/Users/Philipp/git/ba-philipp-doerfler/Thesis/img/Plots/algorithms.pdf"
fig.align_ylabels()
fig.set_size_inches(11.69*4,8.27*4)
fig.savefig(file)